# Task 1

In [67]:
import os
import random
import numpy as np
import pandas as pd
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#from sklearn.impute import SimpleImputer


import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torchsummary import summary
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [3]:
!unzip data_transaction.zip
!unzip data_identity.zip

Archive:  data_transaction.zip
  inflating: data_transaction.csv    
Archive:  data_identity.zip
  inflating: data_identity.csv       


In [4]:
#data_trans = pd.read_csv("C:\\Users\\User\\Downloads\\IU\\AML\\Assignments\\A2\ieee-fraud-detection\data_transaction.csv")
#data_identity = pd.read_csv("C:\\Users\\User\\Downloads\\IU\\AML\\Assignments\\A2\ieee-fraud-detection\data_identity.csv")
data_trans = pd.read_csv("data_transaction.csv")
data_identity = pd.read_csv("data_identity.csv")



In [5]:
print("data transaction shape", data_trans.shape)
print("data identity shape", data_identity.shape)

data transaction shape (590540, 394)
data identity shape (144233, 41)


#### Prepare Data

In [6]:
#df = data_trans.merge(data_identity, how='left', left_index=True, right_index=True)
# Merge the tables 
df = pd.merge(left=data_trans, right=data_identity, left_on='TransactionID', right_on='TransactionID')

df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,0,86535,15.000,H,2803,100.0,150.0,visa,226.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,0,86549,75.887,C,16496,352.0,117.0,mastercard,134.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,0,86555,16.495,C,4461,375.0,185.0,mastercard,224.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0,86620,30.000,H,1790,555.0,150.0,visa,226.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [7]:
del data_trans
del data_identity

In [8]:
df.shape 

(144233, 434)

In [9]:
list_cat = list(df.select_dtypes('object').columns.values)

In [10]:
# Remove the categorical data
df.drop(df[list_cat], axis = 1, inplace = True)

In [11]:
df.shape

(144233, 403)

In [12]:
# Remove empty columns
emp = [col for col in df.columns if df[col].isnull().values.all()]
df.drop(df[emp], axis = 1, inplace = True)

In [13]:
df.shape

(144233, 390)

In [14]:
y = df['isFraud'].to_numpy()
df.drop('isFraud', axis = 1, inplace = True)

#### Split the dataset

In [ ]:
#df_imp.shape

(144233, 390)

In [ ]:
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
0,2987004.0,0.0,86506.0,50.000,4497.0,514.0,150.0,102.0,420.000000,87.000000,...,166.0,14.237337,542.0,144.0,368.26982,16.002708,12.800927,329.608924,149.070308,32.000000
1,2987008.0,0.0,86535.0,15.000,2803.0,100.0,150.0,226.0,337.000000,87.000000,...,166.0,14.237337,621.0,500.0,368.26982,16.002708,12.800927,329.608924,149.070308,32.000000
2,2987010.0,0.0,86549.0,75.887,16496.0,352.0,117.0,134.0,296.845058,86.198088,...,121.0,14.237337,410.0,142.0,368.26982,16.002708,12.800927,329.608924,149.070308,26.508597
3,2987011.0,0.0,86555.0,16.495,4461.0,375.0,185.0,224.0,296.845058,86.198088,...,225.0,14.237337,176.0,507.0,368.26982,16.002708,12.800927,329.608924,149.070308,26.508597
4,2987016.0,0.0,86620.0,30.000,1790.0,555.0,150.0,226.0,170.000000,87.000000,...,166.0,15.000000,529.0,575.0,368.26982,16.002708,12.800927,329.608924,149.070308,24.000000


In [15]:
df.isnull().sum().sum()

13142930

In [16]:
df_clear = df.notna().astype("int").to_numpy()


In [17]:
mask = df.fillna(0)

In [18]:
sc = StandardScaler()
mask = sc.fit_transform(mask.to_numpy())

In [42]:
full_data = np.hstack([mask, df_clear])

In [43]:
full_data.shape

(144233, 778)

In [44]:
#Splitting 
X_train, X_test, y_train, y_test = train_test_split( full_data, y, test_size=0.2, random_state=123)

In [45]:
X_train.shape

(115386, 778)

#### Define the models

#### Undercomplete AE

In [49]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [50]:
## Undercomplete

class autoencoder(nn.Module):
    def __init__(self, input_size, latent_dim):
      super(autoencoder, self).__init__()
      # Step 1 : Define the encoder 
      # Step 2 : Define the decoder
      # Step 3 : Initialize the weights (optional)
      self.encoder = nn.Sequential(
          nn.Linear(input_size, input_size//2),
          nn.ReLU(True),
          nn.Linear(input_size//2, input_size//3),
          nn.Linear(input_size//3, input_size//4),
          nn.Tanh(),
          nn.Linear(input_size//4, latent_dim)
      )
      self.decoder = nn.Sequential(
          nn.Linear(latent_dim, input_size//4),
          nn.ReLU(True),
          nn.Linear(input_size//4, input_size//3),
          nn.Tanh(),
          nn.Linear(input_size//3, input_size//2)
      )
      self.encoder.apply(self.__init_weights)
      self.decoder.apply(self.__init_weights)
        
    def forward(self, x):
      # Step 1: Pass the input through encoder to get latent representation
      # Step 2: Take latent representation and pass through decoder
      x = self.encoder(x)
      x = self.decoder(x)
      return x
        
    
    def encode(self,input):
      #Step 1: Pass the input through the encoder to get latent representation
      return self.encoder(input)
    
    def decode(self, input):
      return self.decoder(input)
    
    def __init_weights(self,m):
      #Init the weights (optional)
      if type(m) == nn.Linear:
          torch.nn.init.xavier_uniform_(m.weight)
          m.bias.data.fill_(0.01)

In [139]:
full_data.shape

(144233, 778)

In [109]:
# Define training parameters

min_lose = 10
batchSize = 512
learning_rate = 0.001
num_epochs = 5
latent_size = 50
###sample = torch.randn((batchSize,1,64))
AE = autoencoder(full_data.shape[1] , latent_size).to(device)
print(AE)
# print(summary(AE,input_size=(1, 64)))

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(AE.parameters(),lr=learning_rate)

# create DataLoaders
dataLoader_train = DataLoader(TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train)), batch_size=batchSize, shuffle=True)
dataLoader_test = DataLoader(TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test)), batch_size=batchSize, shuffle=True)

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=778, out_features=389, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=389, out_features=259, bias=True)
    (3): Linear(in_features=259, out_features=194, bias=True)
    (4): Tanh()
    (5): Linear(in_features=194, out_features=50, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=50, out_features=194, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=194, out_features=259, bias=True)
    (3): Tanh()
    (4): Linear(in_features=259, out_features=389, bias=True)
  )
)


In [110]:
## Trainning the Autoencoder
ind = full_data.shape[1]//2
for epoch in range(num_epochs):
  epoch_loss = 0.0
  for X in dataLoader_train:
    X = X[0].to(device)
    optimizer.zero_grad()

    # forward
    output = AE(X)
    loss = criterion(output * X[:, ind:], X[:, :ind])
    #print(output.dtype)
    # backward
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
  # log

  print('epoch [{}/{}], train_loss:{:.4f}'.format(epoch + 1, num_epochs, epoch_loss / len(dataLoader_train)))

epoch [1/5], train_loss:0.4633
epoch [2/5], train_loss:0.3043
epoch [3/5], train_loss:0.2612
epoch [4/5], train_loss:0.2407
epoch [5/5], train_loss:0.2237


In [142]:
# Test 
min_undercomplete_loss = 10
top = AE.state_dict()

for epoch in range(num_epochs):
  with torch.no_grad():
    loss = 0
    for X in dataLoader_test:
      X = X[0].to(device)
      
      output = AE(X)
      loss += criterion(output * X[:, ind:], X[:, :ind]).item()
    res = loss / len(dataLoader_test)
    if res < min_undercomplete_loss:
        min_undercomplete_loss = res
        top = AE.state_dict()
    else: 
        AE.load_state_dict(top)
    print('epoch [{}/{}], test_loss:{:.4f}'.format(epoch + 1, num_epochs, res ))

epoch [1/5], test_loss:0.1948
epoch [2/5], test_loss:0.1947
epoch [3/5], test_loss:0.1949
epoch [4/5], test_loss:0.1947
epoch [5/5], test_loss:0.1954


#### Regularized AE 

**Sparse autoencoder**

In [70]:
def regLoss(data):
    AE_children = list(AE.children())
    l = 0
    v = data
    for x in range(len(AE_children)):
        v = F.relu((AE_children[x](v)))
        l += torch.mean(torch.abs(v))
    return l

In [71]:
# Train
for epoch in range(num_epochs):
  epoch_loss = 0.0
  for X in dataLoader_train:
    X = X[0].to(device)
    optimizer.zero_grad()

    # forward
    output = AE(X)
    loss = criterion(output * X[:, ind:], X[:, :ind])
    l1_loss = regLoss(X)
    loss = loss +  l1_loss
    #print(output.dtype)
    # backward
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
  # log
  print('epoch [{}/{}], train_loss:{:.4f}'.format(epoch + 1, num_epochs, epoch_loss / len(dataLoader_train)))

epoch [1/5], train_loss:0.3988
epoch [2/5], train_loss:0.2691
epoch [3/5], train_loss:0.2376
epoch [4/5], train_loss:0.2247
epoch [5/5], train_loss:0.2169


In [116]:
# Test
min_reg_ae = 10
for epoch in range(num_epochs):
  with torch.no_grad():
    loss = 0
    for X in dataLoader_test:
      X = X[0].to(device)
      
      output = AE(X)
      loss += criterion(output * X[:, ind:], X[:, :ind]).item()
    
    print('epoch [{}/{}], test_loss:{:.4f}'.format(epoch + 1, num_epochs, loss / len(dataLoader_test)))
    if (loss / len(dataLoader_test)) < min_reg_ae:
      min_reg_ae = (loss / len(dataLoader_test))

epoch [1/5], test_loss:0.1950
epoch [2/5], test_loss:0.1950
epoch [3/5], test_loss:0.1957
epoch [4/5], test_loss:0.1949
epoch [5/5], test_loss:0.1948


#### Variational AE 

In [105]:
# VAE model
class VAE(nn.Module):
    def __init__(self, input_dim= full_data.shape[1], h_dim= 389, z_dim=5):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, input_dim//2)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return self.fc5(h)
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

model = VAE().to(device)

In [106]:
print(model)


VAE(
  (fc1): Linear(in_features=778, out_features=389, bias=True)
  (fc2): Linear(in_features=389, out_features=5, bias=True)
  (fc3): Linear(in_features=389, out_features=5, bias=True)
  (fc4): Linear(in_features=5, out_features=389, bias=True)
  (fc5): Linear(in_features=389, out_features=389, bias=True)
)


In [107]:
# Train
for epoch in range(num_epochs):
  epoch_loss = 0.0
  for X in dataLoader_train:
    X = X[0].to(device)
    optimizer.zero_grad()

    # forward
    x_reconst, mu, log_var = model(X)
    loss = criterion(x_reconst*X[:, ind:], X[:, :ind])
    kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    loss += kl_div

    #print(output.dtype)
    # backward
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
  # log
  print('epoch [{}/{}], train_loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))

epoch [1/5], train_loss:41.6853
epoch [2/5], train_loss:45.6764
epoch [3/5], train_loss:66.9513
epoch [4/5], train_loss:62.2572
epoch [5/5], train_loss:53.7691


In [118]:
# Test
min_variational_ae = 10
for epoch in range(num_epochs):
  with torch.no_grad():
    loss = 0
    for X in dataLoader_test:
      X = X[0].to(device)
      
      x_reconst, mu, log_var = model(X)
      loss += criterion(x_reconst * X[:, ind:], X[:, :ind]).item()
    
    print('epoch [{}/{}], test_loss:{:.4f}'.format(epoch + 1, num_epochs, loss / len(dataLoader_test)))
    if (loss / len(dataLoader_test)) < min_variational_ae:
      min_variational_ae = (loss / len(dataLoader_test))

epoch [1/5], test_loss:0.9690
epoch [2/5], test_loss:0.9689
epoch [3/5], test_loss:0.9650
epoch [4/5], test_loss:0.9667
epoch [5/5], test_loss:0.9721


### Perfomence of autoencoder

In [121]:
print("Undercomplete AE >> Best loss  ::", min_undercomplete_loss)
print("Regularized  AE >> Best loss  ::", min_reg_ae)
print("Variational AE >> Best loss  ::", min_variational_ae)

Undercomplete AE >> Best loss  :: 0.19475951215677095
Regularized  AE >> Best loss  :: 0.19475986481758586
Variational AE >> Best loss  :: 0.965039593085908


####  Compare with statistical approaches

In [135]:
# Imputing the data with mean 

def generate_data(data, indicator):
  XTrain =  YTrain = mask = []
  for x, t in data:
      x = x.cpu().detach().numpy()
      XTrain.append(x[:, :indicator])
      mask.append(x[:, indicator:])
      YTrain.append(t.cpu().detach().numpy())
  XTrain = np.concatenate(XTrain, axis=0)
  mask = np.concatenate(mask, axis=0)
  YTrain = np.concatenate(YTrain, axis=0)
  XTrain = np.where(mask < 1, np.ma.array(XTrain, mask=(mask < 1)).mean(axis=0), XTrain)
  return XTrain, YTrain


train_x, train_y = generate_data(dataLoader_train, ind)





# Task 2 